# Section 1
## Trouver une solution brute-force

- action : part de l'entreprise
- coût : prix d'achat de l'action
- bénéfice : gain réalisé à la revente de l'action après 2 ans

 3 contraintes :
 1. L'action ne peut être acheté qu'une fois.
 2. Il ne peut être acheté de fraction d'action.
 3. 500 euros maximum peuvent être investis.

 Solution Brute-force : 
   - Générer toutes les combinaisons possibles d'actions.
   - Filtrer celles qui respectent la contrainte de budget.
   - Calculer le bénéfice pour chaque combinaison valide.
   - Sélectionner la combinaison avec le bénéfice maximum.


In [4]:
import csv
from itertools import combinations

def parse_benefit(coast, benefit_str):
    """Convertit le bénéfice en pourcentage vers une valeur absolue"""
    if isinstance(benefit_str, str) and benefit_str.endswith('%'):
        percentage = float(benefit_str.rstrip('%'))
        return (coast * percentage) / 100
    return float(benefit_str)

def read_actions_from_csv(file):
    """Lit les actions depuis un fichier CSV avec gestion des pourcentages"""
    actions = []
    with open(file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            coast = float(row['Coast'])
            benefit = parse_benefit(coast, row['Benefit'])
            actions.append({
                'name': row['Actions'],
                'coast': coast,
                'benefit': benefit
            })
    return actions

def find_best_combination(actions, max_budget=500):
    """Trouve la meilleure combinaison d'actions respectant le budget maximum"""
    best_combination = None
    max_benefit = 0
    n = len(actions)
    
    # Générer et évaluer toutes les combinaisons possibles
    for r in range(n + 1):
        for combo in combinations(actions, r):
            total_coast = sum(action['coast'] for action in combo)
            
            # Vérifier si la combinaison respecte le budget
            if total_coast <= max_budget:
                total_benefit = sum(action['benefit'] for action in combo)
                
                # Mettre à jour si c'est la meilleure combinaison
                if total_benefit > max_benefit:
                    max_benefit = total_benefit
                    best_combination = {
                        'actions': combo,
                        'total_coast': total_coast,
                        'total_benefit': total_benefit
                    }
    
    return best_combination

# Lecture des actions et recherche de la meilleure combinaison
actions = read_actions_from_csv('actions.csv')
best = find_best_combination(actions)

# Affichage du résultat
if best:
    print(f"Best combination:")
    print(f"Total cost: {best['total_coast']:.2f} euros")
    print(f"Total benefit: {best['total_benefit']:.2f} euros")
    print(f"Selected actions:")
    for action in best['actions']:
        print(f"  - {action['name']}: {action['coast']:.2f}€ (benefit: {action['benefit']:.2f}€)")


Best combination:
Total cost: 498.00 euros
Total benefit: 99.08 euros
Selected actions:
  - Action-4: 70.00€ (benefit: 14.00€)
  - Action-5: 60.00€ (benefit: 10.20€)
  - Action-6: 80.00€ (benefit: 20.00€)
  - Action-8: 26.00€ (benefit: 2.86€)
  - Action-10: 34.00€ (benefit: 9.18€)
  - Action-11: 42.00€ (benefit: 7.14€)
  - Action-13: 38.00€ (benefit: 8.74€)
  - Action-18: 10.00€ (benefit: 1.40€)
  - Action-19: 24.00€ (benefit: 5.04€)
  - Action-20: 114.00€ (benefit: 20.52€)


# Section 2
## Optimiser l'agorithme

### Livrables : 
- programme Python optimisé
- présentation PPT avec :
    - analyse algo force brute
    - diagramme/organigramme/pseudocode décrivant la solution optimisé
    - algorithme de la version optimisé et cas limites
    - comparaison des performances en utilisant la notation Big-O, la complexité temporelle et l'analyse de la mémoire

### Quelles optimisations ?




# Section 3
## Backtestez et optimisez

Tester le programmer sur deux datasets et comparer les décisions d'achat.